# Imports

In [ ]:
!pip install yolov5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.5/953.5 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.6 MB/s eta 0:00:00


In [ ]:
import zipfile, os
import os
from torch.utils.data import Dataset
from PIL import Image
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import random
import numpy as np
from tqdm import tqdm

from torchvision import transforms
import torchvision.transforms.functional as TF

import random

import yolov5
import torch
import os
import zipfile

os.environ["PYTHONWARNINGS"] = "ignore::FutureWarning"

import cv2
from pathlib import Path
from tqdm import tqdm
import numpy as np
import shutil
import torch
import warnings

warnings.filterwarnings("ignore", message="`torch.cuda.amp.autocast.*`", category=FutureWarning)


import cv2
from torchvision import transforms
from PIL import Image
from pathlib import Path
from tqdm import tqdm
import torch.nn.functional as F

import cv2
import torch
import time
import torch.nn.functional as F
from torchvision import transforms
from pathlib import Path
from PIL import Image
from tqdm import tqdm


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# Utils

In [ ]:
def parse_plate_from_filename(filename, provinces, alphabets, ads):
    name = os.path.splitext(os.path.basename(filename))[0]
    parts = name.split('-')
    if len(parts) < 6:
        raise ValueError(f"File format not valud: {filename}")

    indices = list(map(int, parts[4].split('_')))
    if len(indices) != 7:
        raise ValueError(f"Wrong indeces number in {filename}")

    try:
        plate = (
            provinces[indices[0]] +
            alphabets[indices[1]] +
            ''.join([ads[i] for i in indices[2:]])
        )
    except IndexError:
        raise ValueError(f"Index out of range in {filename}")

    return plate

In [ ]:
def extract_gt_bbox_from_filename(filename):
    parts = filename.split("-")
    if len(parts) < 3:
        raise ValueError(f"Non valid name file: {filename}")

    bbox_str = parts[2]  # esempio: '154&383_386&473'
    try:
        point1_str, point2_str = bbox_str.split("_")
        x1, y1 = map(int, point1_str.split("&"))
        x2, y2 = map(int, point2_str.split("&"))
    except Exception as e:
        raise ValueError(f"Error in retrieving box information from {filename}: {e}")

    return x1, y1, x2, y2


def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    return interArea / float(boxAArea + boxBArea - interArea + 1e-6)

In [ ]:
def greedy_decode(output, idx_to_char):
    output = output.permute(1, 0, 2)  # [B, T, C]
    decoded = []

    for probs in output:
        pred = torch.argmax(probs, dim=1).tolist()  # [T]
        prev = -1
        seq = []
        for p in pred:
            if p != prev and p != 0:
                seq.append(idx_to_char[p])
            prev = p
        decoded.append("".join(seq))

    return decoded


def compute_accuracy(preds, targets):
  correct_chars = total_chars = correct_plates = 0
  for p, t in zip(preds, targets):
      total_chars += max(len(p), len(t))
      correct_chars += sum(a == b for a, b in zip(p, t))
      if p == t:
          correct_plates += 1
  cer = correct_chars / total_chars
  plate_acc = correct_plates / len(targets)
  return cer, plate_acc

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Vocabulary
provinces = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京", "闽", "赣", "鲁", "豫",
             "鄂", "湘", "粤", "桂", "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新", "警", "学", "O"]
alphabets = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N',
             'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'O']
ads = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P',
       'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3',
       '4', '5', '6', '7', '8', '9', 'O']

vocab = sorted(set(provinces + alphabets + ads))
idx_to_char = ['-'] + sorted(set(provinces + alphabets + ads))
char_to_idx = {char: idx for idx, char in enumerate(idx_to_char)}  # ora 0 = blank
num_classes = len(idx_to_char)

In [ ]:
zip_path = '/content/drive/MyDrive/Dataset_CV/DETECTION/CCPD2019_detection.zip'
extract_path = '/content'


os.makedirs(extract_path, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print('End of processing')

End of processing


In [ ]:
transform_crnn = transforms.Compose([
    transforms.Resize((48, 144)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# Network

In [ ]:
# === MODELLO YOLO ===
yolo_model = torch.hub.load('ultralytics/yolov5', 'custom',
                            path='/content/drive/MyDrive/Computer_vision/YoloWeights_finetune/yoloplate_finetuned/weights/best.pt')
yolo_model.conf = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
yolo_model.to(device)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2025-7-12 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [ ]:
class CRNN(nn.Module):
    def __init__(self, img_height, num_classes):
        super(CRNN, self).__init__()

        # INPUT: [B, 3, 48, 144]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),  # RGB input
            nn.ReLU(),
            nn.MaxPool2d(2, 2),              # [B, 64, 24, 72]

            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),              # [B, 128, 12, 36]

            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(),
            nn.Dropout2d(p=0.2),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), (2, 1), padding=(0, 1)),  # [B, 256, 6, 37]

            nn.Conv2d(256, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),

            nn.Conv2d(512, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), (2, 1), padding=(0, 1))   # [B, 512, 3, 38]
        )

        self.rnn = nn.GRU(input_size=512 * 3, hidden_size=128,
                  num_layers=1, bidirectional=True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(128 * 2, num_classes)


    def forward(self, x):
        conv = self.cnn(x)            # [B, 512, 3, 38]
        b, c, h, w = conv.size()
        conv = conv.permute(3, 0, 1, 2)   # [T=38, B, C, H=3]
        conv = conv.reshape(w, b, c * h) # [T=38, B, 512*3]

        rnn_out, _ = self.rnn(conv)      # [T, B, 512]
        rnn_out = self.dropout(rnn_out)
        output = self.fc(rnn_out)        # [T, B, num_classes]
        return F.log_softmax(output, dim=2)


In [ ]:
# === MODELLO CRNN ===
model_crnn = CRNN(img_height=48, num_classes=num_classes)
model_crnn.load_state_dict(torch.load("/content/drive/MyDrive/Computer_vision/crnn_weights_ccpd2019/best_crnn.pth", map_location=device))
model_crnn.eval().to(device)


CRNN(
  (cnn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Dropout2d(p=0.2, inplace=False)
    (9): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=(2, 2), stride=(2, 1), padding=(0, 1), dilation=1, ceil_mode=False)
    (12): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (16): BatchNorm2d(512, eps=

# Evaluation

In [ ]:
# === DATI ===
base_path = Path("/content/CCPD2019_detection/images/test")
all_preds = []
all_gts = []
n_images = 0
start_time = time.time()

for folder in sorted(base_path.iterdir()):
    if not folder.is_dir():
        continue

    print(f"\nFolder: {folder.name}")
    for img_path in tqdm(list(folder.glob("*.jpg"))):
        img_cv = cv2.imread(str(img_path))
        if img_cv is None:
            continue

        try:
            gt_box = extract_gt_bbox_from_filename(img_path.name)
        except:
            continue

        #YOLO detection
        results = yolo_model(str(img_path))
        if len(results.pred[0]) == 0:
            continue

        x1, y1, x2, y2 = map(int, results.pred[0][0][:4])
        pred_box = (x1, y1, x2, y2)

        iou = compute_iou(gt_box, pred_box)
        if iou < 0.6:
            continue

        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(img_cv.shape[1], x2), min(img_cv.shape[0], y2)
        cropped = img_cv[y1:y2, x1:x2]
        if cropped.size == 0:
            continue

        #CRNN
        img_pil = Image.fromarray(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
        img_tensor = transform_crnn(img_pil).unsqueeze(0).to(device)

        with torch.no_grad():
            logits = model_crnn(img_tensor)
            log_probs = F.log_softmax(logits, dim=2)

            # Decoding
            pred = greedy_decode(log_probs, idx_to_char)[0]

        try:
            label = parse_plate_from_filename(img_path.name, provinces, alphabets, ads)
        except:
            continue

        all_preds.append(pred)
        all_gts.append(label)
        n_images += 1


end_time = time.time()
total_time = end_time - start_time
fps = n_images / total_time if total_time else 0

cer, plate_acc = compute_accuracy(all_preds, all_gts)
print(f"\nInference completed on {n_images} images")
print(f"Total time: {total_time:.2f} seconds")
print(f"FPS: {fps:.2f}")
print(f"Character Accuracy:  {cer:.4f}")
print(f"Sequence Accuracy:      {plate_acc:.4f}")



Folder: ccpd_base


100%|██████████| 10000/10000 [03:53<00:00, 42.81it/s]



Folder: ccpd_blur


100%|██████████| 10000/10000 [03:33<00:00, 46.79it/s]



Folder: ccpd_challenge


...


Folder: ccpd_tilt


100%|██████████| 10000/10000 [03:47<00:00, 44.04it/s]



Folder: ccpd_weather


100%|██████████| 9999/9999 [03:46<00:00, 44.21it/s]



Inference completed on 76489 images
Total time: 1793.35 seconds
FPS: 42.65
Character Accuracy:  0.9295
Sequence Accuracy:      0.7757
